<a href="https://colab.research.google.com/github/viperyzen/TRINIT_CyberDuo_ML01/blob/main/Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
from pickletools import optimize
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation,Dropout
from tensorflow.keras.optimizers import SGD

lemmatizer= WordNetLemmatizer() 
intents= json.loads(open('intents.json').read())
words=[]
classes=[]
documents=[]
ignore_letters=['?','!','.',',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list=nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words=[lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words=sorted(set(words))

classes=sorted(set(classes))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

training=[]
output_empty=[0]*len(classes)

for document in documents:
    bag=[]
    word_patterns=document[0]
    word_patterns=[lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)
    
    output_row=list(output_empty)
    output_row[classes.index(document[1])]=1
    training.append([bag,output_row])
random.shuffle(training)
training= np.array(training)

train_x= list(training[:,0])
train_y=list(training[:,1])

model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]),activation='softmax'))

sgd= SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=['accuracy'])

hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=5,verbose=1)
model.save('chatbot_model.h5',hist)
print("done")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


Epoch 1/200
32/32 [==============================] - 1s 2ms/step - loss: 4.1542 - accuracy: 0.0127
Epoch 2/200
32/32 [==============================] - 0s 3ms/step - loss: 4.0403 - accuracy: 0.0701
Epoch 3/200
32/32 [==============================] - 0s 3ms/step - loss: 4.0082 - accuracy: 0.0701
Epoch 4/200
32/32 [==============================] - 0s 3ms/step - loss: 3.9130 - accuracy: 0.0764
Epoch 5/200
32/32 [==============================] - 0s 3ms/step - loss: 3.7790 - accuracy: 0.1146
Epoch 6/200
32/32 [==============================] - 0s 2ms/step - loss: 3.6245 - accuracy: 0.0892
Epoch 7/200
32/32 [==============================] - 0s 3ms/step - loss: 3.5244 - accuracy: 0.1465
Epoch 8/200
32/32 [==============================] - 0s 2ms/step - loss: 3.4232 - accuracy: 0.1592
Epoch 9/200
32/32 [==============================] - 0s 2ms/step - loss: 3.3467 - accuracy: 0.1592
Epoch 10/200
32/32 [==============================] - 0s 2ms/step - loss: 3.0559 - accuracy: 0.1847
Epoch 11/

In [ ]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model
lemmatizer= WordNetLemmatizer() 
intents= json.loads(open('intents.json').read())

words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))
model=load_model('chatbot_model.model')


def clean_up_sentence(sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
      if word ==w:
        bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow = bag_of_words(sentence)
  res= model.predict(np.array([bow]))[0]
  ERROR_THRESHOLD=0.25
  results=[[i,r] for i,r in enumerate(res) if r> ERROR_THRESHOLD]
  results.sort(key=lambda x:x[1], reverse=True)
  return_list=[]
  for r in results:
    return_list.append({'intent':classes[r[0]],'probability':str(r[1])})
    return return_list
def get_response(intents_list,intents_json):
  tag= intents_list[0]['intent']
  list_of_intents= intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result= random.choice(i['responses'])
      break
  return result
print("GO! Bot is running!")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)

GO! Bot is running!
Coronaviruses or COVID-19 is a large family of viruses which may cause illness in animals or humans.  In humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as Middle East Respiratory Syndrome (MERS) and Severe Acute Respiratory Syndrome (SARS). The most recently discovered coronavirus causes coronavirus disease COVID-19?
Good to see you again
Any time!
Stay aware of the latest information on the COVID-19 outbreak, available on the WHO website and through your national and local public health authority.

You can reduce your chances of being infected or spreading COVID-19 by taking some simple precautions:

1.Regularly and thoroughly clean your hands with an alcohol-based hand rub or wash them with soap and water.

2. Maintain at least 1 metre (3 feet) distance between yourself and anyone who is coughing or sneezing.

3.Avoid touching eyes, nose and mouth.

4. Cover your mouth and nose with

In [2]:
import nltk

In [5]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True